In [ ]:
%pip install nltk swifter opendatasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=b29da3082ef6f6fb6e907d8ed4b5c23eb14317e04e07f0e8af5da0068688af91
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import re
import os
import string
import swifter
import torch
import opendatasets as od
import torch.nn as nn

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download Data

1. Option: download the full dataset and select some percentage. You will need to enter your username and kaggle API token when prompted. You can create an API token at https://www.kaggle.com > settings > API > Create New API Token.

2. Option: load preprocessed data and qrels (you can skip the section for preprocessing document and generating queries below)

In [ ]:
# OPTION 1
dataset = 'https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/'
# Download the data set using opendatasets
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 

In [ ]:
# We can't work with a 9GB file, so read 10% of the data at random
p = 0.1
# keep the header, then take only p% of lines
df = pd.read_csv(
    "./genius-song-lyrics-with-language-information/song_lyrics.csv",
    header=0,
    skiprows=lambda i: i > 0 and random.random() > p
)

In [ ]:
# OPTION 2
import urllib.request

urllib.request.urlretrieve('https://f002.backblazeb2.com/file/ffactory-public/documents.pkl', 'documents.pkl')
urllib.request.urlretrieve('https://f002.backblazeb2.com/file/ffactory-public/qrels-train.pkl', 'qrels-train.pkl')
urllib.request.urlretrieve('https://f002.backblazeb2.com/file/ffactory-public/qrels-test.pkl', 'qrels-test.pkl')

('qrels-test.pkl', <http.client.HTTPMessage at 0x7af5311f24a0>)

In [ ]:
df = torch.load('documents.pkl')
qrels_train = torch.load('qrels-train.pkl')
qrels_test = torch.load('qrels-test.pkl')

In [ ]:
df = df.reset_index()

# Preprocessing

In [ ]:
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps = PorterStemmer()
stopwords_en = set(stopwords.words('english'))

# Select only english songs and songs which contain verses (newlines)
df = df[df["language"] == "en"]
df = df[df["lyrics"].str.contains("\n")]

# Select columns we care about
df = df[["title", "lyrics", "views", "id"]]
df.rename(columns={"id": "doc_id"}, inplace=True)
df.set_index("doc_id", inplace=True)

df_proc = df.copy()
df_proc.rename(columns={"lyrics": "text"}, inplace=True)

# Convert to lowercase
df_proc["text"] = df_proc["text"].str.lower()
# Remove any non-alphanumeric / whitespace characters
df_proc["text"] = df_proc["text"].str.replace(re.compile(r"[^\w\s]"), "", regex=True)
# Remove newlines
df_proc["text"] = df_proc["text"].str.replace("\n", " ", regex=False)
# Remove text between square brackets
df_proc["text"] = df_proc["text"].str.replace(re.compile(r"\[.{0,100}\]"), "", regex=True)
# Split text into words
df_proc["text"] = df_proc["text"].str.rsplit()

KeyError: 'language'

In [ ]:
print(f"number of english songs: {len(df_proc)}")
df_proc.head()

In [ ]:
# Remove stopwords and stem tokens
def remove_stopwords_and_stem(tokens):
    return " ".join([ps.stem(token) for token in tokens if token not in stopwords_en])


df_proc["text"] = df_proc["text"].swifter.apply(remove_stopwords_and_stem)

In [ ]:
# Compare the original lyrics with the tokenized lyrics
(df.iloc[5]["lyrics"][0:100], df_proc.iloc[5]["text"][0:100])

In [ ]:
df_proc.head()

,title,views,text
0,Killa Cam,173166,choru opera steve camron killa cam killa cam c...
1,Can I Live,468624,produc irv gotti intro yeah hah yeah rocafella...
2,Forgive Me Father,4743,mayb caus im eatin bastard fiend grub carri pu...
3,Down and Out,144404,produc kany west brian miller intro camron kan...
4,Fly In,78271,intro ask young boy gon second time around gon...


In [ ]:
# Save as a pickle file
torch.save(df_proc[["title", "text"]], 'documents.pkl')

# Test if the pickle file is saved correctly
df_reloaded = torch.load('documents.pkl')
df_reloaded.head()

,title,text
0,Killa Cam,choru opera steve camron killa cam killa cam c...
1,Can I Live,produc irv gotti intro yeah hah yeah rocafella...
2,Forgive Me Father,mayb caus im eatin bastard fiend grub carri pu...
3,Down and Out,produc kany west brian miller intro camron kan...
4,Fly In,intro ask young boy gon second time around gon...


# Generate Queries

In [ ]:
# Functions to Select Verses
def getFirstVerses(lyricsString, amount):
    verseList = re.split('\n', lyricsString)
    FinalList = [i for i in verseList if (len(i) > 1 and i[0] != '[')]
    return " ".join(FinalList[:amount])


def getFirstVersesOfChorus(lyricsString, amount):
    List = re.split('\n', lyricsString)
    verseList = [i for i in List if len(i) > 1]
    for i in range(len(verseList)):
        if "[Chorus" in verseList[i] or "[Hook" in verseList[i]:
            return " ".join(verseList[i + 1:i + amount + 1])
    return getFirstVerses(lyricsString, amount)


def getRandomVerses(lyricsString, amount):
    verseList = re.split('\n', lyricsString)
    FinalList = [i for i in verseList if (len(i) > 1 and i[0] != '[')]
    rd = random.randint(0, len(FinalList) - amount)
    return " ".join(FinalList[rd:rd + amount])

In [ ]:
# Functions to Degrade message

#Function to create typo by neighbouring letter
NeighbouringKeys = {
    'q': "qwas",
    'w': "qwase",
    'e': "wsedr",
    'r': "edrft",
    't': "rftgy",
    'y': "tgyhu",
    'u': "yhuji",
    'i': "ujiko",
    'o': "ikolp",
    'p': "olp",
    'a': "qwasz",
    's': "wazsxed",
    'd': "sxedcrf",
    'f': "dcrfvtg",
    'g': "fvtgbyh",
    'h': "gbyhnuj",
    'j': "hnujmik",
    'k': "jmikol",
    'l': "kolp",
    'z': "azsx",
    'x': "zsxdc",
    'c': "xdcfv",
    'v': "cfvgb",
    'b': "vgbhn",
    'n': "bhnjm",
    'm': "njmk"
}

englishLetters = NeighbouringKeys.keys()


def typos(text, prob=0.01):
    resultingText = ""

    for letter in text:
        if not letter in englishLetters:
            newLetter = letter
        else:
            if random.random() < prob:
                newLetter = random.choice(NeighbouringKeys[letter])
            else:
                newLetter = letter
        resultingText += newLetter

    return resultingText


#Function to (maybe) invert 2 adjacent letters (do force=True to force it to happen)
def invertAdjacentLetters(text, force=False):
    rd = random.randint(0, len(text) - 2)
    if not force:
        if text[rd] in englishLetters and text[rd + 1] in englishLetters:
            return text[:rd] + text[rd + 1] + text[rd] + text[rd + 2:]
        else:
            return text
    else:
        while not (text[rd] in englishLetters and text[rd + 1] in englishLetters):
            rd = random.randint(0, len(text) - 2)
        return text[:rd] + text[rd + 1] + text[rd] + text[rd + 2:]


#Function to (maybe) remove a letter (do force=True to force it to happen)
def removeLetter(text, force=False):
    rd = random.randint(0, len(text) - 1)
    if not force:
        if text[rd] in englishLetters:
            return text[:rd] + text[rd + 1:]
        else:
            return text
    else:
        while not (text[rd] in englishLetters):
            rd = random.randint(0, len(text) - 1)
        return text[:rd] + text[rd + 1:]


#Function to (maybe) double a letter (do force=True to force it to happen)
def doubleLetter(text, force=False):
    rd = random.randint(0, len(text) - 1)
    if not force:
        if text[rd] in englishLetters:
            return text[:rd + 1] + text[rd] + text[rd + 1:]
        else:
            return text
    else:
        while not (text[rd] in englishLetters):
            rd = random.randint(0, len(text) - 1)
        return text[:rd + 1] + text[rd] + text[rd + 1:]


CommonMisspelling = {
    "absence": ["absense", "absentse", "abcense", "absance"],
    "acceptable": ["acceptible"],
    "their": ["there", "they're"],
    "there": ["their", "they're"],
    "they're": ["their", "there"],
    "your": ["you're"],
    "you're": ["your"]
}


# Add a common misspelling
def addCommonMisspell(text):
    for word in CommonMisspelling.keys():
        if word in text:
            return text.replace(word, random.choice(CommonMisspelling[word]))
    return text


In [ ]:
def generate_qrels(df, n):
    # Add 'weight' column
    max_views = max(df["views"])
    df["weight"] = (df["views"] / max_views) ** 0.5 * 0.5 + 0.1

    df_sampled = df.sample(n // 2, weights='weight')

    def generate_positive_qrel(document):
        text = document['lyrics']
        rd = random.random()
        if rd < 0.6:
            query = getFirstVersesOfChorus(text, random.randint(1, 2))
        elif rd < 0.9:
            query = getFirstVerses(text, random.randint(1, 2))
        else:
            query = getRandomVerses(text, random.randint(1, 2))

        if random.randint(0, 3) == 0:
            query = addCommonMisspell(query)
        query = typos(query)

        for j in range(len(query)):
            rand = random.randint(0, 50)
            if rand == 0:
                query = invertAdjacentLetters(query)
            elif rand == 1:
                query = removeLetter(query)
            if rand == 2:
                query = doubleLetter(query)

        doc_id = document.name
        return pd.Series([query, doc_id, 1], index=['text', 'doc_id', 'relevance'])

    def generate_negative_qrel(positive_qrel):
        original_doc_id = positive_qrel['doc_id']
        negative_doc_id = original_doc_id
        while negative_doc_id == original_doc_id:
            negative_doc_id = df.sample(1).iloc[0].name

        return pd.Series([positive_qrel['text'], negative_doc_id, 0])

    positive_qrels = df_sampled.apply(generate_positive_qrel, axis=1, result_type='expand')
    negative_qrels = positive_qrels.apply(generate_negative_qrel, axis=1, result_type='broadcast')

    return pd.concat([positive_qrels, negative_qrels]).reset_index(drop=True)

In [ ]:
# Save qrels as pickle file
total_qrels = 1000
#total_qrels = 100000
qrels_train = generate_qrels(df, int(total_qrels * 0.9))
torch.save(qrels_train, 'qrels-train.pkl')

qrels_test = generate_qrels(df, int(total_qrels * 0.1))
torch.save(qrels_test, 'qrels-test.pkl')

In [ ]:
qrels_train

,text,doc_id,relevance
0,Yo Interrnatinoal thuggin' nigga,5274699,1
1,A lnog time ago,1602274,1
2,"In thw night time, keep me ou of sight, it's t...",2416243,1
3,2015 wrokig at a museum I heard the words from...,6846275,1
4,Now I will burnn before I sing,1749958,1
...,...,...,...
44995,Hook:,7519925,0
44996,Like when I talk my shit you don’t ever hear m...,5103921,0
44997,"You make me dreqm in color, ooh, oh-oh",5417288,0
44998,(Frennvh Dialogue),6761337,0


# TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

feature = tfidf_vectorizer.get_feature_names_out()

In [ ]:
def retrieve_tf_idf_top_n(query, k):
    query_vec = tfidf_vectorizer.transform([query])

    results = cosine_similarity(tfidf_matrix,query_vec).flatten()

    document_pos_indices = sorted(range(len(results)), key = lambda sub: results[sub])[-k:]
    return list(map(lambda x: df_proc.iloc[x]["doc_id"], document_pos_indices))

In [ ]:
# Validate TF-IDF retrieval success rate

N_Tf_Idf = 500
Score = [0]*N_Tf_Idf
for i in range(10000):
    qrel = qrels_train.iloc[i]
    query = qrel["text"]
    correct_doc_id = qrel["doc_id"]

    tf_idf_doc_ids = retrieve_tf_idf_top_n(query, N_Tf_Idf)

    for j, doc_id in enumerate(tf_idf_doc_ids) :
        if doc_id == correct_doc_id:
            Score[j] +=1
print(Score)

In [ ]:
print(sum(Score[-500:]))
print(sum(Score[-100:]))
print(sum(Score[-50:]))
print(sum(Score[-10:]))

In [ ]:
doc_vector = tfidf_matrix[0].toarray()
#df with words and their tf-idf values
df_tfidf = pd.DataFrame(list(zip(feature, doc_vector.flatten())), columns=['Word', 'TF-IDF'])

df_tfidf = df_tfidf.sort_values(by='TF-IDF', ascending=False)
print(df_tfidf)

            Word    TF-IDF
10769        cam  0.800548
34638      killa  0.588214
56995       sing  0.042899
13152       clap  0.041008
65962        uhh  0.018107
...          ...       ...
24039   foodmart  0.000000
24040  foodstamp  0.000000
24041      fooey  0.000000
24042       foof  0.000000
72003        𝑤𝑎𝑠  0.000000

[72004 rows x 2 columns]


# Word2Vec


In [ ]:
# TODO this doesn't work. Remove from final file?
from gensim.models import Word2Vec

print(df_proc.columns)

In [ ]:
# Extract the tokenized lyrics as a list of lists
corpus = df_proc['text'].apply(lambda x: x.split()).tolist()

# Train Word2Vec model
w2v_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
def average_word_vectors(tokens, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    n_words = 0
    for token in tokens:
        if token in model.wv:
            n_words += 1
            feature_vector = np.add(feature_vector, model.wv[token])
    if n_words > 0:
        feature_vector = np.divide(feature_vector, n_words)
    return feature_vector


df_proc['doc_vectors'] = [average_word_vectors(tokens, w2v_model, 100) for tokens in corpus]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#find similar songs to the first one.
similarity_matrix = cosine_similarity(df_proc['doc_vectors'].tolist(), [df_proc['doc_vectors'].iloc[0]])
similar_songs_indices = np.argsort(similarity_matrix[:, 0])[::-1]

top_similar_songs = df_proc.iloc[similar_songs_indices[1:10]][['title', 'text']]
print(top_similar_songs)
#i am just trying things out rn

In [ ]:
def generate_training_data(corpus, window_size, vocab_size, word_to_index, length_of_corpus, sample=None):
    training_data = []
    training_sample_words = []
    for i, word in enumerate(corpus):

        index_target_word = i
        target_word = word
        context_words = []

        #when target word is the first word
        if i == 0:

            # trgt_word_index:(0), ctxt_word_index:(1,2)
            context_words = [corpus[x] for x in range(i + 1, window_size + 1)]

            #when target word is the last word
        elif i == len(corpus) - 1:

            # trgt_word_index:(9), ctxt_word_index:(8,7), length_of_corpus = 10
            context_words = [corpus[x] for x in range(length_of_corpus - 2, length_of_corpus - 2 - window_size, -1)]

        #When target word is the middle word
        else:

            #Before the middle target word
            before_target_word_index = index_target_word - 1
            for x in range(before_target_word_index, before_target_word_index - window_size, -1):
                if x >= 0:
                    context_words.extend([corpus[x]])

            #After the middle target word
            after_target_word_index = index_target_word + 1
            for x in range(after_target_word_index, after_target_word_index + window_size):
                if x < len(corpus):
                    context_words.extend([corpus[x]])

        trgt_word_vector, ctxt_word_vector = get_one_hot_vectors(target_word, context_words, vocab_size, word_to_index)
        training_data.append([trgt_word_vector, ctxt_word_vector])

        if sample is not None:
            training_sample_words.append([target_word, context_words])

    return training_data, training_sample_words

Based on this code: https://github.com/rahul1728jha/Word2Vec_Implementation/blob/master/Word_2_Vec.ipynb

In [ ]:
# Define parameters
window_size = 5
vocab_size = len(your_vocabulary)
word_to_index = your_word_to_index_dict
length_of_corpus = len(corpus)

# Call the function to generate training data
training_data, training_sample_words = generate_word_similarity_training_data(corpus, window_size, vocab_size, word_to_index,
                                                                              length_of_corpus, sample=None)

word2vec_model = Word2Vec(sentences=your_training_data, vector_size=your_vector_size, window=your_window_size, min_count=1, workers=4)

In [ ]:
def cosine_similarity_word(word1, word2, model):
    vector1 = model.wv[word1]
    vector2 = model.wv[word2]
    similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    return similarity

In [ ]:
similarity_score = cosine_similarity_word('word1', 'word2', word2vec_model)
print(f"Similarity between 'word1' and 'word2': {similarity_score}")

# monoBERT
Based on [github.com/veneres/ltr-emb-analysis](https://github.com/veneres/ltr-emb-analysis)

In [ ]:
from torch import nn
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

ModuleNotFoundError: No module named 'transformers'

In [ ]:
batch_size = 4

print(f"{batch_size=}")

num_epochs = 1

print(f"{num_epochs=}")

num_training_steps = int(1e5)

print(f"{num_training_steps=}")

save_checkpoint_after_steps = 1000

print(f"{save_checkpoint_after_steps=}")

docs_dataset_path = "./documents.pkl"
qrels_dataset_path = "./qrels-train.pkl"

output_dir = "out"

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

batch_size=4
num_epochs=1
num_training_steps=100000
save_checkpoint_after_steps=1000


NameError: name 'torch' is not defined

In [ ]:
# Load model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)


tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
class DatasetLoader(Dataset):
    def __init__(self, tokenizer, documents_path: str, qrels_path: str):
        self.documents = torch.load(documents_path)
        self.qrels = torch.load(qrels_path)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.qrels)

    def get_labels(self):
        return self.qrels[:, 2]

    def __getitem__(self, idx):
        qrel = self.qrels.iloc[idx]

        query_text = qrel[0]
        doc_id = qrel[1]
        relevance = qrel[2]

        doc_text = self.documents.loc[self.documents.index == doc_id].iloc[0]["text"]

        #print(f"{query_text=}")
        #print(f"{doc_text=}")

        tokenized_text = self.tokenizer(
            query_text,
            doc_text,
            return_tensors="pt",
            padding="max_length",
            truncation=True
        )
        tokenized_text = {k: v[0] for k, v in tokenized_text.items()}
        #tensor_qid = torch.tensor(int(query_id))
        #"query_id": tensor_qid,
        tensor_did = torch.tensor(int(doc_id))
        return {**tokenized_text, "labels": relevance, "doc_id": tensor_did}

gen = torch.Generator()
gen.manual_seed(2147483647)

train_dataset = DatasetLoader(tokenizer, docs_dataset_path, qrels_dataset_path)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=gen)

NameError: name 'Dataset' is not defined

In [ ]:
train_dataloader.dataset

NameError: name 'train_dataloader' is not defined

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-6)

num_warmup_steps = 1e4

print(f"{num_warmup_steps=}")

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [ ]:
model.to(device)

model.train()

step_n = 0
window_loss = []
for epoch in range(num_epochs):
    with tqdm(train_dataloader, unit="batch", total=min(num_training_steps, len(train_dataloader))) as tepoch:
        #print(next(iter(tepoch)))

        for batch in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            batch_forward = {k: v.to(device) for k, v in batch.items() if k not in ["doc_id", "labels"]}
            outputs = model(**batch_forward)
            logits = outputs.logits
            cel_w = nn.CrossEntropyLoss().to(device)
            loss_w = cel_w(logits.to(device), batch["labels"].to(device))
            loss_w.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            window_loss.append(loss_w.item())
            if len(window_loss) < 100:
                tepoch.set_postfix(loss="---")
            else:
                tepoch.set_postfix(loss=np.mean(window_loss))
                window_loss = window_loss[1:]
            step_n += 1
            if step_n % save_checkpoint_after_steps == 0:
                output_dir_step = output_dir/str(step_n)
                print(f"Saving checkpoint in: {output_dir_step}")
                model.save_pretrained(output_dir_step)

            if step_n == num_training_steps:
                break

output_dir_step = output_dir / str(step_n)
model.save_pretrained(output_dir_step)

# Pipeline

In [ ]:
# Retrieval pipeline
def retrieve(query, df, model, tokenizer, top_k=10):
    query = query.lower()
    query = re.sub(re.compile(r"[^\w\s]"), "", query)
    query = query.rsplit()
    query = " ".join([ps.stem(token) for token in query if token not in stopwords_en])

    # TF-IDF
    tf_idf_doc_ids = retrieve_tf_idf_top_n(query, 100) # Measured to contain the correct doc in >20% of cases

    # monoBERT
    doc_ids_with_score = []
    for i, doc_id in enumerate(tf_idf_doc_ids):
        doc_text = df.loc[df.index == doc_id].iloc[0]["text"]

        #print(f"{query=}")
        #print(f"{doc_text=}")

        tokenized_text = tokenizer(
            query,
            doc_text,
            return_tensors="pt",
            padding="max_length",
            truncation=True
        )
        tokenized_text = {k: v.to(device) for k, v in tokenized_text.items()}

        model.eval()
        with torch.no_grad():
            outputs = model(**tokenized_text)
            logits = outputs.logits
            softmax = nn.Softmax(dim=1)
            probs = softmax(logits)
            probs = probs[:, 1]
            probs = probs.cpu().numpy()
            doc_ids_with_score.append((doc_id, probs[0]))

        if i % 20 == 0:
            print(f"Compared {i} out of {len(tf_idf_doc_ids)} doc ids via monobert")

    doc_ids_with_score = sorted(doc_ids_with_score, key=lambda x: x[1], reverse=True)

    return [doc_id for doc_id, score in doc_ids_with_score[:top_k]]

In [ ]:
# Evaluate using test set
hitrate_sum = 0
mrr_sum = 0
precision_sum = 0

# Can't compare against the whole test set, that would take too long
testing_range = 300
for i in range(testing_range):
    query = qrels_test.iloc[i]["text"]
    doc_id = qrels_test.iloc[i]["doc_id"]
    relevance = qrels_test.iloc[i]["relevance"]

    top_similar_songs = retrieve(query, df_proc, model, tokenizer, top_k=10)

    if doc_id in top_similar_songs:
        hitrate_sum += 1
        mrr_sum += 1 / (top_similar_songs.index(doc_id) + 1)
        precision_sum += 1 / len(top_similar_songs)


mean_hitrate = hitrate_sum / testing_range
mean_mrr = mrr_sum / testing_range
mean_precision = precision_sum / testing_range

In [ ]:
print(f"mean_hitrate: {(mean_hitrate * 100):.2f}%")
print(f"mean_mrr: {mean_mrr:.4f}%")
print(f"mean_precision: {(mean_precision * 100):.2f}%")